In [1]:
import pandas as pd
import numpy as np
import scipy.optimize as opt

In [2]:
# verifica se em um dado intervalo [a, b] no eixio x de uma função f existe raiz 
def has_root(f, interval_a_b):
    if np.sign(f(interval_a_b[0])) * np.sign(f(interval_a_b[1])) < 0: return True

In [3]:
def abs_err(f, previous_root, root):
    return np.absolute(root - previous_root)

In [4]:
def rel_err(f, previous_root, root):
    return np.absolute(root - previous_root) / np.absolute(root)

In [5]:
def get_root_regula_falsi(f, interval_a_b):
    return interval_a_b[1] - ( ( f(interval_a_b[1]) * (interval_a_b[0] - interval_a_b[1]) ) / ( f(interval_a_b[0]) - f(interval_a_b[1]) ) )

In [6]:
def regula_falsi(f, interval_a_b, xtol=-np.inf, rtol=-np.inf, iterations=10000, precision=np.float64):
    iterations_counter = 0
    absolute_error = np.inf
    relative_error = np.inf
    converged = False
    root = None
    
    if has_root(f, interval_a_b):
        interval_a_b = [precision(interval_a_b[0]), precision(interval_a_b[1])]
        previous_root = root
        root = interval_a_b[0]
        
        while True:
            if (iterations_counter == iterations):
                if (absolute_error <= xtol or relative_error <= rtol):
                    converged = True
                break
            if (absolute_error <= xtol or relative_error <= rtol):
                converged = True
                break
            
            previous_root = root
            root = get_root_regula_falsi(f, interval_a_b)
            iterations_counter += 1
            
            absolute_error = abs_err(f, previous_root, root)
            relative_error = rel_err(f, previous_root, root)
            
            if has_root(f, [interval_a_b[0], root]):
                interval_a_b[1] = root
            elif has_root(f, [root, interval_a_b[1]]):
                interval_a_b[0] = root
            else:
                print("regula_falsi, unable to get more precise")
                break
        
    return {"root": root, "iterations": iterations_counter, "converged": converged}

In [7]:
def get_root_newton_raphson(f, df, a):
    return a - (f(a) / df(a))

In [8]:
def get_root_regula_falsi_optimized(f, interval_a_b):
    a, b = interval_a_b[0], interval_a_b[1]
    return  (a * f(b) - b * f(a)) / (f(b) - f(a))

In [9]:
def swap_a_b(interval_a_b):
    a, b = interval_a_b[0], interval_a_b[1]
    interval_a_b[1] = a
    interval_a_b[0] = b

In [10]:
def root_optimized_regula_falsi(f, df, interval_a_b):
    a, b = interval_a_b[0], interval_a_b[1]
    #return a - ( f(a) / 2*df(a) ) * ( ( f(a) - f(b) + (a - b)*df(a) ) / ( f(a) - f(b) ) )
    return (get_root_newton_raphson(f, df, a) + get_root_regula_falsi_optimized(f, interval_a_b)) / 2

In [11]:
def regula_falsi_optimized(f, df, interval_a_b, xtol=-np.inf, rtol=1e-6, iterations=100, precision=np.float128):
    iterations_counter = 0
    absolute_error = np.inf
    relative_error = np.inf
    converged = False
    root = None
    
    if has_root(f, interval_a_b):
        interval_a_b = [precision(interval_a_b[0]), precision(interval_a_b[1])]
        previous_root = root
        root = interval_a_b[0]
        
        while True:
            if (iterations_counter == iterations):
                if (absolute_error <= xtol or relative_error <= rtol):
                    converged = True
                break
            if (absolute_error <= xtol or relative_error <= rtol):
                converged = True
                break
            
            if df(interval_a_b[0]) == 0:
                print("swaped")
                swap_a_b(interval_a_b)
            
            previous_root = root
            root = root_optimized_regula_falsi(f, df, interval_a_b)
            iterations_counter += 1
            
            absolute_error = abs_err(f, previous_root, root)
            relative_error = rel_err(f, previous_root, root)
            
            sign = f(interval_a_b[0]) * f(interval_a_b[1])

            if sign < 0 and np.absolute(f(interval_a_b[0])) >= np.absolute(f(root)):
                interval_a_b = [root, interval_a_b[0]]
            elif sign > 0:
                if np.absolute(f(root)) < np.absolute(f(interval_a_b[1])):
                    interval_a_b[0] = root
                else:
                    interval_a_b = [interval_a_b[1], root]
            else:
                print("regula_falsi_optimized, unable to get more precise")
                break
                    
    return {"root": root, "iterations": iterations_counter, "converged": converged}

In [12]:
# Ite no. == Número da Iteração, BM == Médodo da Bisseção, R-F == Médodo da Falsa Posição, N-R == Método de Newton, PM == Método Proposto
def generateTable(f, df, interval_a_b, ite):
    dit = {"Ite no.": [], "BM approx. root": [], "R-F approx. root": [], "N–R approx. root": [], "PM approx. root": []}

    for i in range(1, ite):
        dit["Ite no."].append(i)
        dit["BM approx. root"].append(opt.root_scalar(f=f, method='bisect', bracket=interval_a_b, maxiter=i).root)
        dit["R-F approx. root"].append(regula_falsi(f=f, interval_a_b=interval_a_b, iterations=i)["root"])
        dit["N–R approx. root"].append(opt.root_scalar(f=f, fprime=df, method='newton', x0=interval_a_b[0], maxiter=i).root)
        dit["PM approx. root"].append(regula_falsi_optimized(f=f, df=df, interval_a_b=interval_a_b, iterations=i)["root"])
        
    return pd.DataFrame(data=dit).set_index("Ite no.")

Example 3

In [13]:
def f(x): return x * np.exp(x) - np.cos(x)
def df(x): return np.exp(x) + np.exp(x) * x + np.sin(x)
interval_a_b = [0, 1]

print("Table 1")
generateTable(f, df, interval_a_b, 22 + 1)

Table 1


,BM approx. root,R-F approx. root,N–R approx. root,PM approx. root
Ite no.,,,,
1,0.500000,0.314665,1.000000,0.657333
2,0.500000,0.446728,0.653079,0.488635
3,0.500000,0.494015,0.531343,0.516509
4,0.500000,0.509946,0.517910,0.517773
5,0.500000,0.515201,0.517757,0.517757
6,0.515625,0.516922,0.517757,0.517757
7,0.515625,0.517485,0.517757,0.517757
8,0.515625,0.517668,0.517757,0.517757
9,0.517578,0.517728,0.517757,0.517757


Example 4

In [14]:
def f(x): return x * np.log10(x) - 1.2 # np.log10(x) == log(x) na base '10'
def df(x): return np.log10(x) + 1 / np.log(10) # np.log(x) == ln(x) na base 'e'
interval_a_b = [1, 3]

print("Table 2")
generateTable(f, df, interval_a_b, 21 + 1)

Table 2


,BM approx. root,R-F approx. root,N–R approx. root,PM approx. root
Ite no.,,,,
1,2.000000,2.676723,3.763102,3.219912
2,2.500000,2.739199,2.806675,2.693526
3,2.500000,2.740614,2.741031,2.739808
4,2.625000,2.740645,2.740646,2.740648
5,2.687500,2.740646,2.740646,2.740646
6,2.718750,2.740646,2.740646,2.740646
7,2.734375,2.740646,2.740646,2.740646
8,2.734375,2.740646,2.740646,2.740646
9,2.738281,2.740646,2.740646,2.740646
